In [ ]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.0 MB/s eta 0:00:00


In [ ]:
from google_play_scraper import reviews
import pandas as pd

reviews_data, _ = reviews(
    'com.mobile.legends',
    lang='id',
    country='id',
    count=50
)

df = pd.DataFrame(reviews_data)
df.to_csv('reviews.csv', index=False)

In [ ]:
df = pd.read_csv('reviews.csv')
print(df.head())

                               reviewId            userName  \
0  9c6a5441-2111-4d17-b442-93f32c996c07          Rahman Ari   
1  1a07d375-9e1d-4a5e-9465-ff04de996569  M Rasya Alfa rizki   
2  76334a31-d51d-4181-b5ad-4ddd8a508913          Jono Supri   
3  310e1137-2686-4911-befa-689853835d23              ZhenXy   
4  4fefb641-dbe8-4825-9f1f-ad59b8e367b0         Alpi Impass   

                                           userImage  \
0  https://play-lh.googleusercontent.com/a/ACg8oc...   
1  https://play-lh.googleusercontent.com/a-/ALV-U...   
2  https://play-lh.googleusercontent.com/a-/ALV-U...   
3  https://play-lh.googleusercontent.com/a-/ALV-U...   
4  https://play-lh.googleusercontent.com/a-/ALV-U...   

                                             content  score  thumbsUpCount  \
0  Perbaiki draksistem gue capek di lock jadi kal...      1              4   
1  Udah bagus sih sekarang udah GK patah patah tp...      4              0   
2  Ml kontol asuuu ke-i konco Yo koyo ngono asu a.

In [ ]:
import re
import string

def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"\d+", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = text.lower()
    return text

df['cleaned_text'] = df['content'].apply(clean_text)
print(df['cleaned_text'].head())

0    perbaiki draksistem gue capek di lock jadi kal...
1    udah bagus sih sekarang udah gk patah patah tp...
2    ml kontol asuuu kei konco yo koyo ngono asu as...
3         kenapa event m tidak hadir di advance sever 
4    gak klasik gak renk banyak kali dark sistem ny...
Name: cleaned_text, dtype: object


In [ ]:
import nltk
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize

df['tokens'] = df['cleaned_text'].apply(word_tokenize)
print(df['tokens'].head())

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


0    [perbaiki, draksistem, gue, capek, di, lock, j...
1    [udah, bagus, sih, sekarang, udah, gk, patah, ...
2    [ml, kontol, asuuu, kei, konco, yo, koyo, ngon...
3    [kenapa, event, m, tidak, hadir, di, advance, ...
4    [gak, klasik, gak, renk, banyak, kali, dark, s...
Name: tokens, dtype: object


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('indonesian'))
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])
print(df['tokens'].head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0    [perbaiki, draksistem, gue, capek, lock, kalah...
1    [udah, bagus, sih, udah, gk, patah, patah, tp,...
2    [ml, kontol, asuuu, kei, konco, yo, koyo, ngon...
3                    [event, m, hadir, advance, sever]
4    [gak, klasik, gak, renk, kali, dark, sistem, n...
Name: tokens, dtype: object


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['cleaned_text']).toarray()
print(X.shape)

(50, 413)


In [ ]:
def label_sentiment(score):
    if score >= 4:
        return "Positive"
    elif score == 3:
        return "Neutral"
    else:
        return "Negative"

df['sentiment'] = df['score'].apply(label_sentiment)  # Kolom 'score' berisi nilai bintang
print(df['sentiment'].value_counts())

sentiment
Negative    32
Positive    15
Neutral      3
Name: count, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.2, random_state=42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

y_pred_nb = nb_model.predict(X_test)
print("Naive Bayes Classification Report:")
print(classification_report(y_test, y_pred_nb))

Naive Bayes Classification Report:
              precision    recall  f1-score   support

    Negative       0.78      1.00      0.88         7
     Neutral       0.00      0.00      0.00         1
    Positive       1.00      0.50      0.67         2

    accuracy                           0.80        10
   macro avg       0.59      0.50      0.51        10
weighted avg       0.74      0.80      0.75        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.svm import SVC

svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

y_pred_svm = svm_model.predict(X_test)
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))

SVM Classification Report:
              precision    recall  f1-score   support

    Negative       0.88      1.00      0.93         7
     Neutral       0.00      0.00      0.00         1
    Positive       1.00      1.00      1.00         2

    accuracy                           0.90        10
   macro avg       0.62      0.67      0.64        10
weighted avg       0.81      0.90      0.85        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.metrics import accuracy_score

print("Naive Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))

Naive Bayes Accuracy: 0.8
SVM Accuracy: 0.9
